## Overview

This notebook:

- creates a MessageHub instance in Bluemix
- produces some messages using vanilla python
- consumes those messages using vanilla python
- produces some more messages using vanilla python
- consumes those messages using scala spark streaming
- tears down the MessageHub instance

Enter your bluemix ID

In [2]:
from getpass import getpass
ibm_id = raw_input("IBM ID: ")

IBM ID: chris.snow@uk.ibm.com


Enter your bluemix ID password

In [3]:
from getpass import getpass
ibm_id_password = getpass("Password: ")

Password: ········


Setup some variables that this notebook will use

In [4]:
# change this to point to your bluemix organization name
bluemix_organization_name = 'chris.snow@uk.ibm.com'

# change this to point to your bluemix space name
bluemix_space_name = 'dev'

# You may need to change the target_endpoint to:
#
#   https://api.ng.bluemix.net     - for the US South Region
#   https://api.eu-gb.bluemix.net  - for the UK
#   https://api.au-syd.bluemix.net - for Austrailia
target_endpoint = 'https://api.ng.bluemix.net'

# This is the name of the Message Hub service instance that will
# be created for you in Bluemix
messagehub_instance_name = 'my_messagehub'

# This is the name of a topic that will get created for you
messagehub_topic_name = 'my_topic'

# Do you want to delete any existing MessageHub instance with
# messagehub_instance_name for before creating a new one?
delete_messagehub_instance = True

# This is the guid of the Message Hub service plan. I found
# this using:
#
#   cf = CloudFoundryUtil(target_endpoint, ibm_id, ibm_id_password)
#   print(cf.search_plans('message hub'))
mh_service_plan_guid = 'fe959ac5-aa47-43a6-9c58-6fc265ee9b0e'

Install a python utility script from github for interacting with cloud foundry via the API

In [5]:
!pip install --user --upgrade protobuf
!pip install --user --upgrade --quiet git+https://github.com/snowch/cf_utils

Requirement already up-to-date: protobuf in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s85d-88ebffb000cc3e-39ca506ba762/.local/lib/python2.7/site-packages
Requirement already up-to-date: setuptools in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s85d-88ebffb000cc3e-39ca506ba762/.local/lib/python2.7/site-packages (from protobuf)
Requirement already up-to-date: six>=1.9 in /usr/local/src/bluemix_jupyter_bundle.v22/notebook/lib/python2.7/site-packages (from protobuf)


Create a python object for interacting with cloud foundry

In [6]:
from cf_utils import cf_utils
cf = cf_utils.CloudFoundryUtil(target_endpoint, ibm_id, ibm_id_password)

Remove any MessageHub instances with the messagehub_instance_name

In [7]:
if delete_messagehub_instance:
    cf.delete_service(service_instance_name=messagehub_instance_name, force=True)

Create a MessageHub service instance

In [8]:
cf.create_service_instance(
    bluemix_organization_name, 
    bluemix_space_name,
    mh_service_plan_guid,
    messagehub_instance_name,
)

Extract some variables that we will need to work with the MessageHub instance

In [29]:
bootstrap_servers = cf.get_service_credential(messagehub_instance_name, 'kafka_brokers_sasl')
# print(bootstrap_servers)

sasl_username = cf.get_service_credential(messagehub_instance_name, 'user')
# print(sasl_username)

sasl_password = cf.get_service_credential(messagehub_instance_name, 'password')
# print(sasl_password)

api_key = cf.get_service_credential(messagehub_instance_name, 'api_key')
# print(api_key)

kafka_admin_url = cf.get_service_credential(messagehub_instance_name, 'kafka_admin_url')
# print(kafka_admin_url)

kafka_rest_url = cf.get_service_credential(messagehub_instance_name, 'kafka_rest_url')
# print(kafka_rest_url)

with open('messagehub.properties', 'w') as w:
    w.write('bootstrap_servers:{0}\n'.format(','.join(bootstrap_servers)))
    w.write('sasl_username={0}\n'.format(sasl_username))
    w.write('sasl_password={0}\n'.format(sasl_password))
    w.write('api_key={0}\n'.format(api_key))
    w.write('kafka_admin_url={0}\n'.format(kafka_admin_url))
    w.write('kafka_rest_url={0}\n'.format(kafka_rest_url))
    w.write('messagehub_topic_name:{0}\n'.format(messagehub_topic_name))

Create a topic in the MessageHub instance

In [18]:
import requests
import json

data = { 'name' : messagehub_topic_name }
headers = {
    'content-type': 'application/json',
    'X-Auth-Token' : api_key 
}
url = kafka_admin_url + '/admin/topics'

# create the topic (http POST)
response = requests.post(url, headers = headers, data = json.dumps(data))

# verify the topic was created (http GET)
response = requests.get(url, headers = headers, data = json.dumps(data))
print (response.text)

[{"name":"my_topic","partitions":1,"retentionMs":"86400000","markedForDeletion":false}]


Install a python kafka client

In [19]:
!pip install --user --quiet kafka-python

Create a kafka producer

In [20]:
def produce_message(message):
    
    # FIXME:this is a lot of overhead - creating a new KafkaProducer each time
    # a message is produced
    
    from kafka import KafkaProducer
    from kafka.errors import KafkaError
    import ssl

    sasl_plain_username = sasl_username
    sasl_plain_password = sasl_password 

    sasl_mechanism = 'PLAIN'
    security_protocol = 'SASL_SSL'

    # Create a new context using system defaults, disable all but TLS1.2
    context = ssl.create_default_context()
    context.options &= ssl.OP_NO_TLSv1
    context.options &= ssl.OP_NO_TLSv1_1

    producer = KafkaProducer(bootstrap_servers = bootstrap_servers,
                             sasl_plain_username = sasl_plain_username,
                             sasl_plain_password = sasl_plain_password,
                             security_protocol = security_protocol,
                             ssl_context = context,
                             sasl_mechanism = sasl_mechanism,
                             api_version = (0,10))

    producer.send(messagehub_topic_name, message)
    producer.flush()

Send some messages

In [ ]:
produce_message(b'some_raw_bytes_1')
produce_message(b'some_raw_bytes_2')

Create a kafka consumer to consume the messages

In [ ]:
def consume_messages():
    from kafka import KafkaConsumer
    from kafka.errors import KafkaError
    import ssl

    sasl_plain_username = sasl_username
    sasl_plain_password = sasl_password 

    sasl_mechanism = 'PLAIN'
    security_protocol = 'SASL_SSL'

    # Create a new context using system defaults, disable all but TLS1.2
    context = ssl.create_default_context()
    context.options &= ssl.OP_NO_TLSv1
    context.options &= ssl.OP_NO_TLSv1_1

    consumer = KafkaConsumer(messagehub_topic_name,
                             bootstrap_servers = bootstrap_servers,
                             sasl_plain_username = sasl_plain_username,
                             sasl_plain_password = sasl_plain_password,
                             security_protocol = security_protocol,
                             ssl_context = context,
                             sasl_mechanism = sasl_mechanism,
                             api_version = (0,10),
                             consumer_timeout_ms = 10000,
                             auto_offset_reset = 'earliest',
                             group_id = "python_client")

    for message in consumer:
        print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition,
                                              message.offset, message.key,
                                              message.value))

consume_messages()

Now run the notebook **"Step 9 - Scala spark streaming on DSX"** and while running, put some more messages in kafka

In [80]:
produce_message('1,500') # user_id | movie_id
produce_message('1,501') # user_id | movie_id
produce_message('1,502') # user_id | movie_id
produce_message('1,503') # user_id | movie_id
produce_message('100000,503') # user_id | movie_id : invalid user

Remove the MessageHub instance from Bluemix

In [ ]:
if delete_messagehub_instance:
    cf.delete_service(service_instance_name=messagehub_instance_name, force=True)